# **Chapter 13. Naive Bayes**

## **Bayes' Theorem**

* In probability theory and statistics, **Bayes' theorem** describes the probability of an event, based on prior knowledge of conditions that might be related to the event.
$$ P(A|B) = \frac{P(B|A)P(A)}{P(B)}$$
Alternative form:
$$ P(A|B) = \frac{P(B|A)P(A)}{P(B|A)P(A) + P(B|\neg A)P(\neg A)} $$

## **A Really Dumb Spam Filter**

* Let $S$ be the event "the message is spam" and $B$ be the event "the message contains the word bitcoin." Bayes's theorem tells us that the probability that the message is spam conditional on containing the word bitcoin is:
$$P (S|B) = \frac{P (B | S) P (S)}{P (B | S) P (S) + P (B | \neg S) P (\neg S)}$$
>* The numerator is the probability that a message is spam _and_ contains _bitcoin_, while the denominator is just the probability that a message contains _bitcoin_. Hence, you can think of this calculation as simply representing the proportion of _bitcoin_ messages that are spam.

* If we have a large collection of messages we know are spam, and a large collection of messages we know are not spam, then we can easily estimate $P(B|S)$ and $P(B|\neg S)$. If we further assume that any message is equally likely to be spam or not spam (so that $P(S) = P(\neg S) = 0.5$), then:
$$P (S | B) = \frac{P (B | S)}{P (B | S) + P (B | \neg S)}$$

* For example, if 50% of spam messages have the word bitcoin, but only 1% of nonspam messages do, then the probability that any given bitcoin-containing email is spam is: $\frac{0.5}{0.5+0.01} =98\%$.


## **A More Sophisticated Spam Filter**

* Suppose that we have a vocabulary of many words, $w_1, \ldots, w_n$. To move this into the realm of probability theory, we'll write $X_i$ for the event "a message contains the word $w_i$." Also imagine that we've come up with an estimate $P(X_i|S)$ for the probability that a spam message contains the $i$-th word, and a similar estimate $P(X_i|\neg S)$ for the probability that a nonspam message contains the $i$-th word.

* The key to Naive Bayes is making the assumption that the presences (or absences) of each word are **independent of one another**, conditional on a message being spam or not. In math terms, this means that:
$$P(X_1=x_1,\ldots,X_n=x_n | S) =P(X_1=x_1 | S) \times \cdots \times P(X_n=x_n | S)$$

* This is an extreme assumption. (There’s a reason the technique has **naive** in its name.) Despite the unrealisticness of this assumption, this model often performs well and has historically been used in actual spam filters.

* The same Bayes's theorem reasoning we used for our "bitcoin-only" spam filter tells us that we can calculate the probability a message is spam using the equation:
$$ P (S | X = x) = \frac{P(X = x | S)}{P (X = x | S) + P (X = x | \neg S)}$$

* The Naive Bayes assumption allows us to compute each of the probabilities on the right simply by multiplying together the individual probability estimates for each vocabulary word.

* In practice, you usually want to avoid multiplying lots of probabilities together, to prevent a problem called *underflow*, in which computers don't deal well with floating-point numbers that are too close to 0. Recalling from algebra that $\log (ab) = \log a + \log b$ and that $\exp(\log x) = x$, we usually compute $p_1 \times \cdots \times p_n$ as the equivalent (but floating-point-friendlier): $\exp (\log p_1 + \cdots + \log p_n)$.

* The only challenge left is coming up with estimates for $P (X_i | S)$ and $P(X_i | \neg S)$, the probabilities that a spam message (or nonspam message) contains the word $w_i$. If we have a fair number of training messages labeled as spam and not spam, an obvious first try is to estimate $P(X_i | S)$ simply as the fraction of spam messages containing the word $w_i$.

* Suppose that in our training set the vocabulary word *data* only occurs in nonspam messages. Then we'd estimate $P( \text{data} | S) = 0$. The result is that our Naive Bayes classifier would always assign spam probability $0$ to any message containing the word *data*, even a message like "data on free bitcoin and authentic rolex watches." To avoid this problem, we usually use some kind of smoothing.
>* Refer to **Laplace smoothing** at this [link](https://towardsdatascience.com/laplace-smoothing-in-na%C3%AFve-bayes-algorithm-9c237a8bdece).

* In particular, we'll choose a pseudocount—$k$—and estimate the probability of seeing the $i$-th word in a spam message as:
$$P (X_i | S) = \frac{k + \text{number of spams containing}~w_i}{2k + \text{number of spams}}$$

* For example, if *data* does **not** occur in $98$ spam messages, and if $k$ is 1, we estimate $P(\text{data}|S)$ as $\frac{1}{100} = 0.01$, which allows our classifier to still assign some nonzero spam probability to messages that contain the word *data*.

## **Implementation**

In [ ]:
from typing import Set
import re

def tokenize(text: str) -> Set[str]:
    text = text.lower()                         # Convert to lowercase,
    all_words = re.findall("[a-z0-9']+", text)  # extract the words, and
    return set(all_words)                       # remove duplicates.

assert tokenize("Data Science is science") == {"data", "science", "is"}

In [ ]:
from typing import NamedTuple

class Message(NamedTuple):
    text: str
    is_spam: bool

* Ultimately, we will want to predict $P(S | token)$.
* _probabilities(): To apply Bayes's theorem we need to know $P(token | S)$ and $P(token | \neg S)$ for each token in the vocabulary.
* predict(): we will sum up the log probabilities.

In [ ]:
from typing import List, Tuple, Dict, Iterable
import math
from collections import defaultdict

class NaiveBayesClassifier:
    def __init__(self, k: float = 0.5) -> None:
        self.k = k  # smoothing factor

        self.tokens: Set[str] = set()
        self.token_spam_counts: Dict[str, int] = defaultdict(int)
        self.token_ham_counts: Dict[str, int] = defaultdict(int)
        self.spam_messages = self.ham_messages = 0

    def train(self, messages: Iterable[Message]) -> None:
        for message in messages:
            # Increment message counts
            if message.is_spam:
                self.spam_messages += 1
            else:
                self.ham_messages += 1

            # Increment word counts
            for token in tokenize(message.text):
                self.tokens.add(token)
                if message.is_spam:
                    self.token_spam_counts[token] += 1
                else:
                    self.token_ham_counts[token] += 1

    def _probabilities(self, token: str) -> Tuple[float, float]:
        """returns P(token | spam) and P(token | not spam)"""
        spam = self.token_spam_counts[token]
        ham = self.token_ham_counts[token]

        p_token_spam = (spam + self.k) / (self.spam_messages + 2 * self.k)
        p_token_ham = (ham + self.k) / (self.ham_messages + 2 * self.k)

        return p_token_spam, p_token_ham

    def predict(self, text: str) -> float:
        text_tokens = tokenize(text)
        log_prob_if_spam = log_prob_if_ham = 0.0

        # Iterate through each word in our vocabulary.
        for token in self.tokens:
            prob_if_spam, prob_if_ham = self._probabilities(token)

            # If *token* appears in the message,
            # add the log probability of seeing it;
            if token in text_tokens:
                log_prob_if_spam += math.log(prob_if_spam)
                log_prob_if_ham += math.log(prob_if_ham)

            # otherwise add the log probability of _not_ seeing it
            # which is log(1 - probability of seeing it)
            else:
                log_prob_if_spam += math.log(1.0 - prob_if_spam)
                log_prob_if_ham += math.log(1.0 - prob_if_ham)

        prob_if_spam = math.exp(log_prob_if_spam)
        prob_if_ham = math.exp(log_prob_if_ham)
        return prob_if_spam / (prob_if_spam + prob_if_ham)

## **Testing Our Model**

In [ ]:
messages = [Message("spam rules", is_spam=True),
            Message("ham rules", is_spam=False),
            Message("hello ham", is_spam=False)]

model = NaiveBayesClassifier(k=0.5)
model.train(messages)

In [ ]:
assert model.tokens == {"spam", "ham", "rules", "hello"}
assert model.spam_messages == 1
assert model.ham_messages == 2
assert model.token_spam_counts == {"spam": 1, "rules": 1}
assert model.token_ham_counts == {"ham": 2, "rules": 1, "hello": 1}

In [ ]:
text = "hello spam"

* $P(\text{"spam"} | S) \times P(\neg \text{"ham"} | S) \times P(\neg \text{"rules"} | S) \times P(\text{"hello"} | S)$

In [ ]:
probs_if_spam = [
    (1 + 0.5) / (1 + 2 * 0.5),      # "spam"  (present)
    1 - (0 + 0.5) / (1 + 2 * 0.5),  # "ham"   (not present)
    1 - (1 + 0.5) / (1 + 2 * 0.5),  # "rules" (not present)
    (0 + 0.5) / (1 + 2 * 0.5)       # "hello" (present)
]

* $P(\text{"spam"} | \neg S) \times P(\neg \text{"ham"} | \neg  S) \times P(\neg \text{"rules"} | \neg  S) \times P(\text{"hello"} | \neg  S)$

In [ ]:
probs_if_ham = [
    (0 + 0.5) / (2 + 2 * 0.5),      # "spam"  (present)
    1 - (2 + 0.5) / (2 + 2 * 0.5),  # "ham"   (not present)
    1 - (1 + 0.5) / (2 + 2 * 0.5),  # "rules" (not present)
    (1 + 0.5) / (2 + 2 * 0.5),      # "hello" (present)
]

In [ ]:
p_if_spam = math.exp(sum(math.log(p) for p in probs_if_spam))
p_if_ham = math.exp(sum(math.log(p) for p in probs_if_ham))

# Should be about 0.83
assert model.predict(text) == p_if_spam / (p_if_spam + p_if_ham)

## **Using Our Model**

* A popular (if somewhat old) dataset is the SpamAssassin public corpus. We'll look at the files prefixed with *20021010*.

In [ ]:
from io import BytesIO # So we can treat bytes as a file.
import requests # To download the files, which
import tarfile # are in .tar.bz format.

BASE_URL = "https://spamassassin.apache.org/old/publiccorpus"
FILES = ["20021010_easy_ham.tar.bz2",
         "20021010_hard_ham.tar.bz2",
         "20021010_spam.tar.bz2"]

# This is where the data will end up,
# in /spam, /easy_ham, and /hard_ham subdirectories.
# Change this to where you want the data.
OUTPUT_DIR = 'spam_data'

for filename in FILES:
    # Use requests to get the file contents at each URL.
    content = requests.get(f"{BASE_URL}/{filename}").content
    # Wrap the in-memory bytes so we can use them as a "file."
    fin = BytesIO(content)
    # And extract all the files to the specified output dir.
    with tarfile.open(fileobj=fin, mode='r:bz2') as tf:
        tf.extractall(OUTPUT_DIR)

In [ ]:
import glob, re
    
# modify the path to wherever you've put the files
path = 'spam_data/*/*'
    
data: List[Message] = []
    
# glob.glob returns every filename that matches the wildcarded path
for filename in glob.glob(path):
    is_spam = "ham" not in filename
    
    # There are some garbage characters in the emails, the errors='ignore'
    # skips them instead of raising an exception.
    with open(filename, errors='ignore') as email_file:
        for line in email_file:
            if line.startswith("Subject:"):
                subject = line.lstrip("Subject: ")
                data.append(Message(subject, is_spam))
                break  # done with this file

In [ ]:
import requests
url = 'https://raw.githubusercontent.com/joelgrus/data-science-from-scratch/master/scratch/machine_learning.py'
r = requests.get(url)

with open('machine_learning.py', 'w') as f:
    f.write(r.text)

import random
from machine_learning import split_data
    
random.seed(0)      # just so you get the same answers as me
train_messages, test_messages = split_data(data, 0.75)
    
model = NaiveBayesClassifier()
model.train(train_messages)

In [ ]:
from collections import Counter
    
predictions = [(message, model.predict(message.text))
               for message in test_messages]
    
# Assume that spam_probability > 0.5 corresponds to spam prediction
# and count the combinations of (actual is_spam, predicted is_spam)
confusion_matrix = Counter((message.is_spam, spam_probability > 0.5)
                            for message, spam_probability in predictions)
    
print(confusion_matrix)

Counter({(False, False): 674, (True, True): 89, (True, False): 37, (False, True): 25})


* This gives 89 true positives (spam classified as "spam"), 25 false positives (ham classified as "spam"), 674 true negatives (ham classified as "ham"), and 37 false negatives (spam classified as "ham"). This means our precision is $\frac{89}{89 + 25} = 78\%$, and our recall is $\frac{89}{89 + 37} = 71\%$, which are not bad numbers for such a simple model.

In [ ]:
def p_spam_given_token(token: str, model: NaiveBayesClassifier) -> float:
    # We probably shouldn't call private methods, but it's for a good cause.
    prob_if_spam, prob_if_ham = model._probabilities(token)
    
    return prob_if_spam / (prob_if_spam + prob_if_ham)
    
words = sorted(model.tokens, key=lambda t: p_spam_given_token(t, model))
    
print("spammiest_words", words[-10:])
print("hammiest_words", words[:10])

spammiest_words ['clearance', 'sale', '95', 'per', 'zzzz', 'only', 'money', 'systemworks', 'rates', 'adv']
hammiest_words ['spambayes', 'users', 'zzzzteana', 'razor', 'sadev', 'ouch', 'apt', 'spamassassin', 'spam', 'bliss']
